In [1]:
import numpy as np
import matplotlib as plt
import seaborn as sns
import pandas as pd

The data comes from : https://hbiostat.org/data/repo/titanic3.xls

In [ ]:
data = pd.read_excel('tit.xlsx', index_col=0)  